In [2]:
# print("""
# ```{=html}
#       </br>
#       <table class='table table-striped' id='mapTable'>
#       <thead>
#       <tr>
#       <th style="width:35%"><b onclick="sortTable(0)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Name</th>
#       <th><b onclick="sortTable(1)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
#       <th style="width:10%"><b onclick="sortTable(2)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Straße</th>
#       <th><b onclick="sortTable(3)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Ort</th>
#       <th><b onclick="sortTable(4)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Web</th>
#       <th><b onclick="sortTable(5)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Gründung</th>
#       </tr>
#       </thead>
#       <tbody>
# ```
#       """)
# for name, lat, lon, website, item, place, street, year in locations:
#   print(f"""
# ```{{=html}}
#   <tr>
#   <td><span style='display:none'>{name}</span><a href='{item}'>{name}</a></td>
# ```
#   """)
#   if lat == '' and lon == '':
#     print("""
# ```{=html}
#   <td></td>
# ```
# """)
#   else:
#     print(f"""
# ```{{=html}}
#   <td>{lat},<br> {lon}</td>
# ```
# """)
#   print(f"""
# ```{{=html}}
# <td>{street}</td>
# ```
# """)
#   print(f"""
# ```{{=html}}
#   <td>{place}</td>
# ```
# """)
#   if website == '':
#     print("""
# ```{=html}
# <td></td>
# ```
# """)
#   else:
#     print(f"""
# ```{{=html}}
# <td><a style="text-decoration:none;color:#df382c;font-size:big" class="bi bi-link-45deg" href="{website}"></a></td>
# ```
# """)
#   print(f"""
# ```{{=html}}
# <td>{year}</td>
# ```
# """)
#   print("</tr>")
# print("""
# ```{=html}
# </tbody></table>
# ```
# """)


# print("""
# <script>
# function sortTable(n) {
#   var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0;
#   table = document.getElementById("mapTable");
#   switching = true;
#   // Set the sorting direction to ascending:
#   dir = "asc";
#   /* Make a loop that will continue until
#   no switching has been done: */
#   while (switching) {
#     // Start by saying: no switching is done:
#     switching = false;
#     rows = table.rows;
#     /* Loop through all table rows (except the
#     first, which contains table headers): */
#     for (i = 1; i < (rows.length - 1); i++) {
#       // Start by saying there should be no switching:
#       shouldSwitch = false;
#       /* Get the two elements you want to compare,
#       one from current row and one from the next: */
#       x = rows[i].getElementsByTagName("TD")[n];
#       y = rows[i + 1].getElementsByTagName("TD")[n];
#       /* Check if the two rows should switch place,
#       based on the direction, asc or desc: */
#       if (dir == "asc") {
#         if (x.innerHTML.toLowerCase() > y.innerHTML.toLowerCase()) {
#           // If so, mark as a switch and break the loop:
#           shouldSwitch = true;
#           break;
#         }
#       } else if (dir == "desc") {
#         if (x.innerHTML.toLowerCase() < y.innerHTML.toLowerCase()) {
#           // If so, mark as a switch and break the loop:
#           shouldSwitch = true;
#           break;
#         }
#       }
#     }
#     if (shouldSwitch) {
#       /* If a switch has been marked, make the switch
#       and mark that a switch has been done: */
#       rows[i].parentNode.insertBefore(rows[i + 1], rows[i]);
#       switching = true;
#       // Each time a switch is done, increase this count by 1:
#       switchcount ++;
#     } else {
#       /* If no switching has been done AND the direction is "asc",
#       set the direction to "desc" and run the while loop again. */
#       if (switchcount == 0 && dir == "asc") {
#         dir = "desc";
#         switching = true;
#       }
#     }
#   }
# }
# </script>
# """) 

In [3]:
import requests, folium, time
from folium import plugins
from folium.plugins import BeautifyIcon

COMMONS_API = "https://commons.wikimedia.org/w/api.php"

def get_files(category, limit=20):
    r = requests.get(COMMONS_API, params={
      "action":"query","format":"json",
      "list":"categorymembers","cmtitle":f"Category:{category}",
      "cmlimit":limit,"cmtype":"file","nocache":str(time.time())
    })
    return r.json()["query"]["categorymembers"]

def get_coords(title):
    r = requests.get(COMMONS_API, params={
      "action":"query","titles":title,"prop":"coordinates",
      "colimit":"1","format":"json","nocache":str(time.time())
    }).json()
    for p in r["query"]["pages"].values():
        if "coordinates" in p:
            return p["coordinates"][0]["lat"], p["coordinates"][0]["lon"]
    return None,None

def get_url(title):
    r = requests.get(COMMONS_API, params={
      "action":"query","titles":title,"prop":"imageinfo",
      "iiprop":"url","format":"json"
    }).json()
    for p in r["query"]["pages"].values():
        return p["imageinfo"][0]["url"]

files = get_files("HSH_Kurs_2025", limit=20)
locations = []
for f in files:
    lat, lon = get_coords(f["title"])
    if lat is not None:
        locations.append((f["title"], lat, lon, get_url(f["title"])))
    time.sleep(0.2)

if not locations:
    print("No structured geocoordinates found.")
else:
    m = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=14)

for idx, (name, lat, lon, img) in enumerate(locations, start=1):
    # clean up the label
    label = name.replace('File:', '').rsplit('.', 1)[0]

    # build the popup as before
    popup_html = f"""
    <h4 style="text-align: center; font-weight:bold">{label}</h4><br>
    <img src="{img}" style="
        width: 200px;      /* fixed square width */
        height: 200px;     /* fixed square height */
        object-fit: cover; /* crop/scale to fill */
        border-radius: 100px;
        display: block;
        margin: auto;
    ">
    """

number = 0
print('<ol>')

for name, lat, lon, img in locations:
    # remove the File: prefix
    base = name.replace('File:', '')
    # split on the last dot, keep the part before it
    clean = base.rsplit('.', 1)[0]
    popup_html = f"""
    <h4 style="text-align: center; font-weight:bold">{clean}</h4><br>
    <img src="{img}" style="
        width: 200px;      /* fixed square width */
        height: 200px;     /* fixed square height */
        object-fit: cover; /* crop/scale to fill */
        border-radius: 100px;
        display: block;
        margin: auto;
    ">
    """

    number = number + 1

    print(f'<li>{clean}</li>')

    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_html, max_width=220),
        icon=BeautifyIcon(
            icon_shape='marker', number=number, background_color='lightgreen'
        ),
        tooltip=clean
    ).add_to(m)

print('</ol>')

folium.plugins.Fullscreen(
    position="topright",
    title="Vollbild",
    title_cancel="Schließen",
    force_separate_button=True,
).add_to(m)

m

<ol>
<li>Kurt - Schwitters - Platz road sign</li>
<li>Stahl 5-81 Erich Hauser 2</li>
<li>Stahl 5-81 Erich Hauser</li>
</ol>
